Aggregation and enrichment
==============
This notebook puts all of the geojson files in the `geoJson` directory together in a single feature collection.  
While it's doing that, it's also assigning new properties to the individual features.  
These features include:  
* collection: Collection name (for the book it's part of)
* lat_extent: number of degrees that the map is "tall".
* lng_extent: number of degrees that the map is "wide".

In [1]:
import os, json

In [2]:
fileList = os.listdir("geoJson/")
if "all_atlases.geojson" in fileList:
    fileList.pop(fileList.index("all_atlases.geojson"))

In [4]:
aggregator = {"crs": {"type": "name","properties": {"name": "EPSG:4326"}},"type":"FeatureCollection","features":[]}
numfeat = 0
for f in fileList:
    with open("geoJson/"+f, "r") as fp:
        jsonFile = json.load(fp)
    features = jsonFile['features']
    numfeat += len(features)
    for fe in features:
        colname = f
        colname = colname.replace("_fp.geojson","")
        colname = colname.replace("fp.geojson","no_collection")
        fe['properties']['collection'] = colname
        lats = [g[1] for g in fe['geometry']["coordinates"][0]]
        lngs = [g[0] for g in fe['geometry']["coordinates"][0]]
        lat_extent = max(lats) - min(lats)
        lng_extent = max(lngs) - min(lngs)
        fe['properties']['lat_extent'] = lat_extent
        fe['properties']['lng_extent'] = lng_extent
        fe['properties']['degree_area'] = lat_extent * lng_extent
    aggregator['features'].extend(features)

In [10]:
aggregator['features'] = sorted(aggregator['features'],key=lambda k: k['properties']['degree_area'],reverse=True)

In [11]:
numfeat

471

In [12]:
len(aggregator['features'])

471

In [13]:
with open("geoJson/all_atlases.geojson","w") as fp:
    json.dump(aggregator,fp,indent=2,sort_keys=True)